# MTH 651: Advanced Numerical Analysis

## Homework Assignment 5

### (Student Name 👈 Write Your Name Here!)

### Guidelines

* Each student must complete their own assignment individually.
  * Discussing with other students is allowed (encouraged!), but you must write your own answers and not copy off of others.
* Submit the homework as a Jupyter notebook with **properly formatted LaTeX**


#### Problem 1. (2 points)

(Exercise 4.x.1 from the textbook). Prove that
$$
    D^\alpha_x T^m_y u(x) = T^{m - |\alpha|}_y D^\alpha_x u(x)
$$
for $u \in C^{|\alpha|}$ with $|\alpha| < m$, and hence
$$
    D^\alpha Q^m_B u(x) = Q^{m - |\alpha|}_B D^\alpha u(x).
$$

#### Problem 2. (2 points)

(Exercise 4.x.2 from the textbook). Prove the version of Taylor's theorem with the integral form of the remainder: given $f \in C^m([0,1])$,
$$
    f(1) = \sum_{k=0}^{m-1} \frac{1}{k!} f^{(k)}(0) + m \int_0^1 \frac{1}{m!} s^{m-1}f^{(m)}(1-s)\, ds.
$$

Hint: Induction:

1. Prove the base case for $m=1$.
2. Assuming the formula holds for $m-1$, prove that it holds for $m$ (integration by parts).

#### Problem 3. (2 points)

Recall that the mass matrix can be used to compute the $L^2$ projection onto a finite element space.

Given $u \in C^{k+1}(\Omega)$, let $u_h$ be its $L^2$ projection.

Derive and prove error estimates for $\| u - u_h \|_{L^2(\Omega)}$.

Hint: Bramble-Hilbert and Galerkin orthogonality/optimality.

#### Problem 4. Coding. (4 points)

Perform convergence studies for the $L^2$ projection to confirm (or falsify) the estimates from Problem 3.

First, write a function `compute_l2_projection_error` that computes and returns the $L^2$ error of the $L^2$ projection of the given function $f$.
Adapt the code at the bottom of this file to compute the L2 projection and return the error in the $L^2$ norm.

In [ ]:
def compute_l2_projection_error(n, k, f):
    """
    l2_projection_error(n, k, f)

    Computes the L2 projection of f onto an order-k finite element space. The
    input parameter n denotes the mesh size.

    Return the L2 error of the projection compared with the input function.
    """

Use `compute_l2_projection_error` to perform convergence studies using $k=1$, $k=2$, and $k=3$ using the function $f$ defined below.

Print out tables of the $L^2$ errors and compute the rates.

**Hint:** Use/adapt the functions `convergence_study` and `print_convergence_table` from the [Assignment 3 Partial Solutions](https://github.com/pazner/mth-651/blob/main/homework_assignments/assignment_3_partial_solutions.ipynb) to help with this.

Do the results agree with the theory?

In [ ]:
%pip install mfem
%pip install glvis

In [ ]:
import mfem.ser as mfem
from math import cos, exp, sin

class f(mfem.PyCoefficient):
    def EvalValue(self, xvec):
        x = xvec[0]
        y = xvec[1]
        return exp(0.1*sin(5.1*x - 6.2*y) + 0.3*cos(4.3*x +3.4*y))

Adapt this code to write your function `compute_l2_projection_error`:

In [ ]:
# Create the mesh and finite element space
mesh = mfem.Mesh(10, 10, "TRIANGLE")
order_k = 1
fe_collection = mfem.H1_FECollection(order_k, mesh.Dimension())
fe_space = mfem.FiniteElementSpace(mesh, fe_collection)

# 5. Define the solution x as a finite element grid function in fespace. Set
#    the initial guess to zero, which also sets the boundary conditions.
u = mfem.GridFunction(fe_space)

# Create the right-hand side vector using the linear form F : v -> (f, v)
# This is called a "domain linear form integrator", or DomainLFIntegrator
F = mfem.LinearForm(fe_space)
F.AddDomainIntegrator(mfem.DomainLFIntegrator(f()))
# F.Assemble() tells MFEM to actually compute the entries of the vector F
F.Assemble()

# Set up the bilinear form m(.,.). The L2 inner product is called a
# "mass integrator"
m = mfem.BilinearForm(fe_space)
m.AddDomainIntegrator(mfem.MassIntegrator())
m.Assemble()
m.Finalize()

# Get the matrix associated with the bilinear form
M = m.SpMat()
GS = mfem.GSSmoother(M)
u.Assign(0.0)
mfem.PCG(M, GS, F, u, 0, 200, 1e-16, 0.0)

u.ComputeL2Error(f())